# LLMath Demo

This notebook demonstrates the theorem-aware mathematical proof assistant.

Components:
- **NaturalProofs Retrieval**: Semantic search over mathematical theorems
- **SymPy Tools**: Symbolic computation (simplify, solve, diff, integrate)
- **DeepSeek-Math**: Fine-tuned LLM for proof-style answers

## Setup

First, install the package and dependencies.

In [ ]:
# Install from the parent directory
!pip install -e ..

In [ ]:
import sys
sys.path.insert(0, '../src')

from llmath.config import LLMathConfig
from llmath.retrieval import NaturalProofsRetriever
from llmath.retrieval.theorem_kb import TheoremKB
from llmath.tools import simplify_expr, solve_equation, differentiate_expr, integrate_expr
from llmath.prompts import build_math_prompt, build_baseline_prompt

## 1. Retrieval: NaturalProofs Search

The retriever builds a FAISS index over the NaturalProofs dataset for semantic search.

In [ ]:
# Initialize retriever (builds index on first run)
retriever = NaturalProofsRetriever(
    index_path='../data/naturalproofs_faiss.index',
    meta_path='../data/naturalproofs_meta.json',
    rebuild_index=False,
)

In [ ]:
# Test retrieval
query = "continuous function differentiable at a point"
results = retriever.search(query, k=3)

print(f"Query: {query}")
print()
for i, r in enumerate(results, 1):
    print(f"[{i}] score={r['score']:.3f}")
    print(f"    {r['text'][:200]}...")
    print()

## 2. TheoremKB: Formatted Snippets

The TheoremKB wraps the retriever to produce formatted snippets with titles.

In [ ]:
kb = TheoremKB(retriever)

question = "Show that the sum of two continuous functions is continuous."
theorems = kb.get_theorems(question, k=3)

print(f"Question: {question}")
print()
for i, t in enumerate(theorems, 1):
    print(f"[T{i}] {t.title} (score={t.score:.3f})")
    print(f"    {t.snippet[:150]}...")
    print()

## 3. SymPy Tools: Symbolic Computation

The tools module provides wrappers around SymPy for common operations.

In [ ]:
# Simplification
print("simplify((x**2 - 1) / (x - 1)) =", simplify_expr("(x**2 - 1) / (x - 1)"))

# Solving equations
print("solve(x**2 - 1 = 0) =", solve_equation("x**2 - 1 = 0"))

# Differentiation
print("diff(x**2 * sin(x)) =", differentiate_expr("x**2 * sin(x)"))

# Integration
print("integrate(2*x) =", integrate_expr("2*x"))

## 4. Prompt Building

The prompt builder combines retrieved theorems and SymPy results into a structured prompt.

In [ ]:
question = "Prove that the derivative of x**2*sin(x) is 2*x*sin(x) + x**2*cos(x)."
theorems = kb.get_theorems(question, k=3)
sympy_context = ["diff(x**2*sin(x), x) -> 2*x*sin(x) + x**2*cos(x)"]

prompt = build_math_prompt(question, theorems, sympy_context)

print("Generated prompt:")
print("=" * 60)
print(prompt[:1500])
print("...")

## 5. Full Agent Pipeline

The MathAgent combines all components for end-to-end inference.

**Note**: This requires a GPU with sufficient VRAM to load DeepSeek-Math.

In [ ]:
# Uncomment to run the full agent (requires GPU)
# from llmath.agent import create_math_agent
#
# agent = create_math_agent(retriever)
#
# result = agent.run(
#     question="Prove that the derivative of x**2*sin(x) is 2*x*sin(x) + x**2*cos(x).",
#     sympy_expressions=["diff: x**2*sin(x)"],
# )
#
# print(result)

## 6. Gradio Demo

Launch the interactive web interface.

In [ ]:
# Uncomment to launch Gradio demo
# from llmath.api.gradio_app import launch_demo
# launch_demo(share=True)